# Testing speed of the tree algorithm

# Implement a simple network

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

sys.path.append('../')

from src_experiment import NeuralNet
from geobin import RegionTree, TreeNode

In [2]:
hidden_sizes = [3,3,3,3]
num_classes = 2
net = NeuralNet(
    input_size = 2,
    hidden_sizes = hidden_sizes,
    num_classes = num_classes,
)
state_dict = net.state_dict()

In [3]:
state_dict

OrderedDict([('l1.weight',
              tensor([[-0.0953, -0.2715],
                      [ 0.2344,  0.2789],
                      [ 0.5666,  0.1944]])),
             ('l1.bias', tensor([0.2169, 0.5188, 0.4811])),
             ('l2.weight',
              tensor([[ 0.4524,  0.5452, -0.1802],
                      [ 0.0561, -0.2465, -0.4239],
                      [-0.3039,  0.0836,  0.5201]])),
             ('l2.bias', tensor([ 0.4890, -0.3448,  0.5718])),
             ('l3.weight',
              tensor([[-0.0937,  0.3103,  0.1525],
                      [ 0.2433, -0.4311, -0.5513],
                      [ 0.0700, -0.2935, -0.4506]])),
             ('l3.bias', tensor([-0.2411, -0.2519, -0.4104])),
             ('l4.weight',
              tensor([[ 0.4074, -0.1942, -0.2127],
                      [-0.5416,  0.1409,  0.4546],
                      [-0.5644, -0.2567,  0.4897]])),
             ('l4.bias', tensor([-0.0762, -0.3481, -0.2517])),
             ('l5.weight',
              tenso

In [4]:
# Without feasibility check
start = time.time()
tree = RegionTree(state_dict)
tree.build_tree(verbose=True, check_feasibility=False)
end = time.time()
time_taken = end-start


Building tree...


Layer 5: 100%|██████████| 4/4 [00:00<00:00,  9.86it/s]


In [5]:
print(f"Hidden layers: {hidden_sizes}")
print(f"Output: {num_classes}")
theoretical_max = 2**np.cumsum(np.append(np.array(hidden_sizes), num_classes))
print(f"Theoretical max size:    {(np.append(1, theoretical_max))}")
print(f"Actual size of tree:         {np.array(tree.size)}")


Hidden layers: [3, 3, 3, 3]
Output: 2
Theoretical max size:    [    1     8    64   512  4096 16384]
Actual size of tree:         [    1     8    64   512  4096 16384]


In [6]:
def check_speed_comparison(hidden_sizes, num_classes, check=False):
    net = NeuralNet(
    input_size = 2,
    hidden_sizes = hidden_sizes,
    num_classes = num_classes,
    )
    state_dict = net.state_dict()
    
    # Standard way
    start = time.time()
    tree = RegionTree(state_dict)
    tree.build_tree(verbose=False, check_feasibility=check)
    end = time.time()
    time_taken = end-start

    print("--------------------------------------")
    print(f"Checking feasibility: {str(check)}")
    print(f"Hidden layers: {hidden_sizes}")
    print(f"Output: {num_classes}")
    theoretical_max = 2**np.cumsum(np.append(np.array(hidden_sizes), num_classes))
    print(f"Theoretical max size:   {(np.append(1, theoretical_max))}")
    print(f"Actual size of tree:    {np.array(tree.size)}")
    print(f"Time taken: {time_taken:.3f}s")
    print("--------------------------------------")

    
def compare_equal_configs(hidden_sizes, num_classes):
    check_speed_comparison(hidden_sizes, num_classes, check=False)
    check_speed_comparison(hidden_sizes, num_classes, check=True)

In [7]:
compare_equal_configs(
    hidden_sizes=[7,5,3],
    num_classes=1
)

--------------------------------------
Checking feasibility: False
Hidden layers: [7, 5, 3]
Output: 1
Theoretical max size:   [    1   128  4096 32768 65536]
Actual size of tree:    [    1   128  4096 32768 65536]
Time taken: 2.047s
--------------------------------------
--------------------------------------
Checking feasibility: True
Hidden layers: [7, 5, 3]
Output: 1
Theoretical max size:   [    1   128  4096 32768 65536]
Actual size of tree:    [ 1 29 60 95 95]
Time taken: 1.353s
--------------------------------------


In [8]:
check_speed_comparison(
    hidden_sizes=[5,5,5],
    num_classes=1,
    check=True
)

--------------------------------------
Checking feasibility: True
Hidden layers: [5, 5, 5]
Output: 1
Theoretical max size:   [    1    32  1024 32768 65536]
Actual size of tree:    [ 1 16 59 76 88]
Time taken: 1.972s
--------------------------------------


## Testing speed of dataloaders

In [9]:
from src_experiment import moon_path, wbc_path, get_moons_data, get_wbc_data
import torch
_, data = get_wbc_data(label_noise=0.0, batch_size=32)

# test params
experiment = "wbc"
arch = "decreasing"
dropout = 0.0
noise = 0.0
run_number = 0
path_fn = moon_path if experiment == "moon" else wbc_path
data_fn = get_moons_data if experiment == "moon" else get_wbc_data

# Load the state dicts
epochs = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,74]
state_dicts = {epoch: path_fn(arch, dropout, noise, run_number) /"state_dicts"/f"epoch{epoch}.pth" for epoch in epochs}

In [10]:
def check_speed_comparison_real(check=False):

    # Standard way
    start = time.time()
    state_dict = torch.load(state_dicts[0])
    tree = RegionTree(state_dict)
    tree.build_tree(verbose=False, check_feasibility=check)
    intermediate = time.time()
    
    tree.pass_dataloader_through_tree(data)
    tree.collect_number_counts()
    counts = tree.get_number_counts()
    
    end = time.time()
    time_taken = end-start

    print("----------Build and passing----------")
    print(f"Checking feasibility: {str(check)}")
    # print(f"Hidden layers: {hidden_sizes}")
    # print(f"Output: {num_classes}")
    # # theoretical_max = 2**np.cumsum(np.append(np.array(hidden_sizes), num_classes))
    # print(f"Theoretical max size:   {(np.append(1, theoretical_max))}")
    # print(f"Actual size of tree:    {np.array(tree.size)}")
    print(f"Total time taken:    {time_taken:.3f} s")
    print(f"Time building:       {intermediate-start:.3f} s")
    print(f"Time passing:        {end-intermediate:.3f} s")
    print("--------------------------------------")

In [ ]:
check_speed_comparison_real(
    check=True,
)

----------Build and passing----------
Checking feasibility: True
Total time taken:    2.454 s
Time building:       2.444 s
Time passing:        0.010 s
--------------------------------------


In [12]:
compare_equal_configs(
    hidden_sizes=[7,5,3],
    num_classes=2
)

--------------------------------------
Checking feasibility: False
Hidden layers: [7, 5, 3]
Output: 2
Theoretical max size:   [     1    128   4096  32768 131072]
Actual size of tree:    [     1    128   4096  32768 131072]
Time taken: 3.500s
--------------------------------------
--------------------------------------
Checking feasibility: True
Hidden layers: [7, 5, 3]
Output: 2
Theoretical max size:   [     1    128   4096  32768 131072]
Actual size of tree:    [  1  29  74 140 140]
Time taken: 1.750s
--------------------------------------
